<a href="https://colab.research.google.com/github/DPariser/DataScience/blob/main/QC_and_Pre_Processing_FASTQ_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single-cell RNA-seq data processing
** Information found in publication: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7857060/

Single-cell sequencing data were aligned and quantified using kallisto/bustools (KB, v0.24.4) (Bray et al., 2016 [link text](https://www.nature.com/articles/nbt.3519)) against the GRCh38 human reference genome downloaded from 10x Genomics official website. Preliminary counts were then used for downstream analysis. Quality control was applied to cells based on three metrics step by step: the total UMI counts, number of detected genes and proportion of mitochondrial gene counts per cell. Specifically, cells with less than 1000 UMI counts and 500 detected genes were filtered, as well as cells with more than 10% mitochondrial gene counts. To remove potential doublets, for PBMC samples, cells with UMI counts above 25,000 and detected genes above 5,000 are filtered out. For other tissues, cells with UMI counts above
70,000 and detected genes above 7,500 are filtered out. Additionally, we applied Scrublet (Wolock et al., 2019 [link text](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6625319/pdf/nihms-1515604.pdf)) to identify potential
doublets. The doublet score for each single cell and the threshold based on the bimodal distribution was calculated using default
parameters. The expected doublet rate was set to be 0.08, and cells predicted to be doublets or with doubletScore larger than  0.25 were filtered. After quality control, a total of 1,598,708 cells were remained. The stepwise quality control metrics used for indi-
vidual samples were listed in Table S1. The resulting distribution of UMI counts, gene counts as well as mitochondrial gene percent-
age were shown in Figures S1C–S1E. We normalized the UMI counts with the deconvolution strategy implemented in the R package scran. Specifically, cell-specific size factors were computed by computeSumFactors function and further used to scale the counts for
each cell. Then the logarithmic normalized counts were used for the downstream analysis.

# Batch effect correction and cell subsets annotations

To integrate cells into a shared space from different datasets for unsupervised clustering, we used the harmony algorithm ([Korsunsky
et al., 2019][link text](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6884693/pdf/nihms-1539299.pdf)) to do batch effect correction. To detect the most variable genes used for harmony algorithm, we performed variable gene selection separately for each sample. A consensus list of 1,500 variable genes was then formed by selecting the genes with the greatest recovery rates across samples, with ties broken by random sampling. All ribosomal, mitochondrial and immunoglobulin genes were then removed from the list. Next, we calculate a PCA matrix with 20 components using such informative genes and then
feed this PCA matrix into HarmonyMatrix() function implemented in R package Harmony. We set sample and dataset as two technical covariates for correction with theta set as 2.5 and 1.5, respectively. The resulting batch-corrected matrix was used to build nearest neighbor graph using Scanpy (Wolf et al., 2018 [link text](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5802054/pdf/13059_2017_Article_1382.pdf)). Such nearest neighbor graph was then used to find clusters by Louvain algorithm
(Traag et al., 2019 [link text](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6435756/pdf/41598_2019_Article_41695.pdf)). The cluster-specific marker genes were identified using the rank_genes_groups function.
The first round of clustering (resolution = 0.3) identified six major cell types including T cells, NK cells, B cells, plasma B cells,
myeloid cells and epithelial cells. To identify clusters within each major cell type, we performed a second round of clustering on
T/NK, B/plasma B, myeloid and epithelial cells separately. The procedure of the second round of clustering is the same as first round,
starting from low-rank harmony output (30 components) on the highly variable genes chosen as described above, with resolution
ranging from 0.3 to 1.5. Each sub cluster was restrained to have at least 30 significantly highly expressed genes (FDR < 0.01, logFC > 0.25, t test) compared with other cells. Annotation of the resulting clusters to cell types was based on the known markers. Mean-while, single cells expressing two sets of well-studied canonical markers of major cell types were labeled as doublets and excluded from the following analysis. Also, cells highly expressed HBA, HBB and HBD, which are the markers for erythrocytes, were also
excluded. 136,006 cells were removed and a total of 1,462,702 cells were retained for downstream analysis. In total, we identified 6 major cell types including B cells (MS4A1, CD79A, CD79B), myeloid cells (CST3, LYZ), NK cells (GNLY, NKG7, TYROBP), epithelial cells (KRT18, KRT19), CD4 and CD8 T cells (CD3D, CD3E, CD3G, CD40LG, CD8A, CD8B). These major cell types were further classified into 64 clusters representing different cell types within major cell lineages (Figures 1B and S1F–S1J). A full list of canonical and signature marker genes for each cluster was deposited in Table S2.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas numpy scikit-learn htseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Unzip Files
For each patient there are two gz files and a .xml file the gz files include the raw sequencing files R1 refers to read 1 and R2 refers to read two, which during our later processing steps we will need to ensure that they align properly and the .xml file is the metadata information such as sample identifiers, library preparation protocols, sequencing platforms, and other relevant details.

In [ ]:
import gzip
import shutil

# path to input files
fastq1_gz = '/content/drive/MyDrive/path/to/fastq1.gz'
fastq2_gz = '/content/drive/MyDrive/path/to/fastq2.gz'
xml_gz = '/content/drive/MyDrive/path/to/xml.gz'

# path to output files
fastq1 = '/content/drive/MyDrive/path/to/fastq1'
fastq2 = '/content/drive/MyDrive/path/to/fastq2'
xml = '/content/drive/MyDrive/path/to/xml'

# extract fastq1.gz
with gzip.open(fastq1_gz, 'rb') as f_in:
    with open(fastq1, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# extract fastq2.gz
with gzip.open(fastq2_gz, 'rb') as f_in:
    with open(fastq2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# extract xml.gz
with gzip.open(xml_gz, 'rb') as f_in:
    with open(xml, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

FileNotFoundError: ignored

# Quality Control
## Load Long Ranger and GRCh38 Human Genome Data

1.   Single-cell sequencing data were aligned and quantified using kallisto/bustools (KB, v0.24.4) (Bray et al., 2016) against the GRCh38 human reference genome downloaded from 10x Genomics official website.

Install instructions can be found here: https://support.10xgenomics.com/genome-exome/software/pipelines/latest/installation

Code for the GRCh38: https://support.10xgenomics.com/genome-exome/software/downloads/latest?

The GRCh38 reference genome is a widely used reference genome for human sequencing because it represents the most current and accurate version of the human genome. It includes the latest updates and revisions, including new genome sequences and gene annotations, and provides improved coverage of difficult-to-sequence regions such as centromeres and telomeres. It is also used as a reference for many large-scale genomics projects, such as the Human Genome Project, the 1000 Genomes Project, and the Genotype-Tissue Expression (GTEx) project.


In [18]:
!cd /opt

In [19]:
# Download and unpack the Long Ranger file
# Long Ranger - 2.2.2 (March 26, 2018)
!wget -O longranger-2.2.2.tar.gz "https://cf.10xgenomics.com/releases/genome/longranger-2.2.2.tar.gz?Expires=1678942097&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvZ2Vub21lL2xvbmdyYW5nZXItMi4yLjIudGFyLmd6IiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjc4OTQyMDk3fX19XX0_&Signature=GHLpJcQ6WIza~wstIxoVHGaEvVAPZfuCP~VbmRb6PuZzqcNMfNeViiKQfx~JpqNpEXKv-eUyDpkyapH5~eWOVDQ09irzJjwNb1JATeo-FWwGBOOVR1ps2A-eVWkDPbbHbkdi2snHKGawL1ZogGm-DRkCqqGfTiGdAh7sXYHGb-3v4eWDtKgiG6icf202HvQSM8oSnZdQftvwp20EkY0Np5M6VH16-dL3RKN0zVqn3scTRFW4gdGJwyQQep1Y8IdNVrgaxEzAM2WkWutJ0zKgTwMW9ODS1dnSQGzOaYY5NF9OWbAwE36gSffBi~Y-CJO058KmFpnrwbNkbP0ztg8HsA__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA"

--2023-03-15 16:53:23--  https://cf.10xgenomics.com/releases/genome/longranger-2.2.2.tar.gz?Expires=1678942097&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvZ2Vub21lL2xvbmdyYW5nZXItMi4yLjIudGFyLmd6IiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjc4OTQyMDk3fX19XX0_&Signature=GHLpJcQ6WIza~wstIxoVHGaEvVAPZfuCP~VbmRb6PuZzqcNMfNeViiKQfx~JpqNpEXKv-eUyDpkyapH5~eWOVDQ09irzJjwNb1JATeo-FWwGBOOVR1ps2A-eVWkDPbbHbkdi2snHKGawL1ZogGm-DRkCqqGfTiGdAh7sXYHGb-3v4eWDtKgiG6icf202HvQSM8oSnZdQftvwp20EkY0Np5M6VH16-dL3RKN0zVqn3scTRFW4gdGJwyQQep1Y8IdNVrgaxEzAM2WkWutJ0zKgTwMW9ODS1dnSQGzOaYY5NF9OWbAwE36gSffBi~Y-CJO058KmFpnrwbNkbP0ztg8HsA__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 490894203 (468M) [application/x-tar]
S

In [20]:
# Download and unpack the Long Ranger file
# Long Ranger - 2.2.2 (March 26, 2018)
!tar -xzvf longranger-2.2.2.tar.gz

Streaming output truncated to the last 5000 lines.
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/server/tests/test_client.py
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/server/tests/test_server.py
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/server/tests/test_server.pyc
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/server/server.pyc
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/server/__init__.pyc
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/sql.py
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/pytables.pyc
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/compatibility.pyc
longranger-2.2.2/anaconda-cs/2.2.0-anaconda-cs-c8/lib/python2.7/site-packages/blaze/mongo.pyc
longranger-2.2.2/anaco

In [25]:
# Download and unpack the reference data file
# GRCh38 Reference - 2.1.0 (Sep 15, 2016)
!wget https://cf.10xgenomics.com/supp/genome/refdata-GRCh38-2.1.0.tar.gz

--2023-03-15 17:11:13--  https://cf.10xgenomics.com/supp/genome/refdata-GRCh38-2.1.0.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5187997538 (4.8G) [application/x-tar]
Saving to: ‘refdata-GRCh38-2.1.0.tar.gz’

refdata-GRCh38-2.1. 100%[===================>]   4.83G  56.7MB/s    in 3m 14s  

2023-03-15 17:14:28 (25.5 MB/s) - ‘refdata-GRCh38-2.1.0.tar.gz’ saved [5187997538/5187997538]



In [26]:
# Download and unpack the reference data file
# GRCh38 Reference - 2.1.0 (Sep 15, 2016)
!tar -xzvf refdata-GRCh38-2.1.0.tar.gz

refdata-GRCh38-2.1.0/
refdata-GRCh38-2.1.0/version
refdata-GRCh38-2.1.0/README.BEFORE.MODIFYING
refdata-GRCh38-2.1.0/fasta/
refdata-GRCh38-2.1.0/fasta/genome.dict
refdata-GRCh38-2.1.0/fasta/genome.fa
refdata-GRCh38-2.1.0/fasta/genome.fa.amb
refdata-GRCh38-2.1.0/fasta/genome.fa.ann
refdata-GRCh38-2.1.0/fasta/genome.fa.bwt
refdata-GRCh38-2.1.0/fasta/genome.fa.fai
refdata-GRCh38-2.1.0/fasta/genome.fa.flat
refdata-GRCh38-2.1.0/fasta/genome.fa.gdx
refdata-GRCh38-2.1.0/fasta/genome.fa.pac
refdata-GRCh38-2.1.0/fasta/genome.fa.sa
refdata-GRCh38-2.1.0/fasta/primary_contigs.txt
refdata-GRCh38-2.1.0/fasta/sex_chromosomes.tsv
refdata-GRCh38-2.1.0/genes/
refdata-GRCh38-2.1.0/genes/gene_annotations.gtf.gz
refdata-GRCh38-2.1.0/genome
refdata-GRCh38-2.1.0/regions/
refdata-GRCh38-2.1.0/regions/centromeres.tsv
refdata-GRCh38-2.1.0/snps/


In [27]:
# Prepend the Long Ranger directory to your $PATH. This will allow you to invoke the longranger commands.
!export PATH=/opt/longranger-2.2.2:$PATH

In [28]:
# Site check
# Next, please run the bundled site check script and send the output to 10x. We will review the information to ensure that Long Ranger will run smoothly once you have generated your own Chromium data. Assuming you have installed and entered the 10x environment as described above, please run the following commands:
!longranger sitecheck > sitecheck.txt
!longranger upload dpariser@mit.edu sitecheck.txt

/bin/bash: longranger: command not found
/bin/bash: longranger: command not found


In [29]:
# Verify Installation
# To ensure that the longranger pipeline is installed correctly, use longranger testrun. This test can take up to 60 minutes on a sixteen-core workstation. Assuming you have installed Long Ranger into /opt, the command to run the test would look like:
!export PATH=/opt/longranger-2.2.2:$PATH
!longranger testrun --id=tiny

/bin/bash: longranger: command not found


In [30]:
!longranger upload dpariser@mit.edu tiny/tiny.mri.tgz

/bin/bash: longranger: command not found


## Aligning and quantifying using kallisto/bustool

Kallisto is a program for quantifying abundances of transcripts from RNA-Seq data, which uses a novel idea of pseudoalignment for fast and accurate quantification of transcript abundances from RNA-Seq data. Kallisto can quantify expression levels of genes, transcripts, and isoforms. Kallisto generates an index from the reference transcriptome that allows fast pseudoalignment of RNA-Seq reads, followed by generation of gene- and transcript-level counts.

Bustools is a set of tools for analyzing BUS files generated by kallisto. BUS files contain information about which barcodes were detected in which transcript and how many UMIs were associated with each barcode-transcript pair. Bustools can be used to correct and sort the barcode and UMI information in the BUS file, filter out low-quality reads and barcodes, count the number of unique molecular identifiers (UMIs) associated with each gene or transcript, and perform other downstream analyses.

We will need to reference the GRCh38 Human Genome here to align it to our sequencing data

In [32]:
!sudo apt-get install kallisto

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhts3
The following NEW packages will be installed:
  kallisto libhts3
0 upgraded, 2 newly installed, 0 to remove and 22 not upgraded.
Need to get 598 kB of archives.
After this operation, 1,637 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libhts3 amd64 1.10.2-3ubuntu0.1 [350 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 kallisto amd64 0.46.1+dfsg-2build1 [248 kB]
Fetched 598 kB in 1s (561 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debco

In [33]:
!git clone https://github.com/BUStools/bustools.git
!cd bustools && ./compile.sh

Cloning into 'bustools'...
remote: Enumerating objects: 3362, done.
remote: Counting objects: 100% (569/569), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 3362 (delta 425), reused 460 (delta 397), pack-reused 2793
Receiving objects: 100% (3362/3362), 3.34 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (1309/1309), done.
/bin/bash: ./compile.sh: No such file or directory


In [ ]:
# Install kallisto and bustools
!pip install kallisto
!pip install bustools

# Download and unpack the reference data file
!wget https://cf.10xgenomics.com/supp/genome/refdata-gex-GRCh38-2020-A.tar.gz
!tar -xzvf refdata-gex-GRCh38-2020-A.tar.gz

# Align reads to the reference genome using kallisto
!kallisto bus -i /content/refdata-gex-GRCh38-2020-A/kallisto_index.idx -o output/ -x 10xv3 -t 2 /content/drive/MyDrive/Colab_Notebooks/Lung_Mk/HRR339742_f1.fastq /content/drive/MyDrive/Colab_Notebooks/Lung_Mk/HRR339742_f2.fastq

# Correct and sort the barcode and UMIs & generates a matrics with UMI counts/gene for each barvode in the sorted BUS file
!bustools correct -w /content/refdata-gex-GRCh38-2020-A/10xv3_whitelist.txt -p output/output.bus | bustools sort -T output/tmp/ -t 2 -p - | bustools count -o output/counts -g /content/refdata-gex-GRCh38-2020-A/genes/genes.gtf -e output/matrix.ec -t output/transcripts.txt --genecounts -



## Filtering cells based on count
Preliminary counts were then used for downstream analysis. Quality control was applied to cells based on three metrics step by step: the total UMI counts, number of detected genes and proportion of mitochondrial gene counts per cell. Specifically, cells with less than 1000 UMI counts and 500 detected genes were filtered, as well as cells with more than 10% mitochondrial gene counts. 

In [ ]:
import pandas as pd

# Load the output from bustools count
matrix = pd.read_csv("output/counts/bus_output/output.bus.count.txt", sep="\t", index_col=0, header=None, skiprows=1)

# Calculate the total UMI counts and number of detected genes per cell
umi_counts = matrix.sum(axis=1)
gene_counts = (matrix > 0).sum(axis=1)

# Load the gene annotation file
genes = pd.read_csv("refdata-gex-GRCh38-2020-A/genes/genes.gtf", sep="\t", comment="#", header=None)

# Extract mitochondrial gene names
mito_genes = genes[genes[0] == "MT"][8].str.extract(r'gene_name "(.+?)"', expand=False)

# Calculate the proportion of mitochondrial gene counts per cell
mito_counts = matrix[matrix.index.isin(mito_genes)].sum(axis=0)
mito_prop = mito_counts / umi_counts

# Filter cells with less than 1000 UMI counts, less than 500 detected genes, or more than 10% mitochondrial gene counts
cells_to_keep = (umi_counts >= 1000) & (gene_counts >= 500) & (mito_prop <= 0.1)
filtered_matrix = matrix[cells_to_keep]

# Calculate total UMI counts and number of detected genes for the filtered matrix
filtered_umi_counts = filtered_matrix.sum(axis=1)
filtered_gene_counts = (filtered_matrix > 0).sum(axis=1)


## Remove potential doublets

To remove potential doublets, for PBMC samples, cells with UMI counts above 25,000 and detected genes above 5,000 are filtered out. For other tissues, cells with UMI counts above 70,000 and detected genes above 7,500 are filtered out. Additionally, we applied Scrublet (Wolock et al., 2019 link text) to identify potential doublets. The doublet score for each single cell and the threshold based on the bimodal distribution was calculated using default parameters. The expected doublet rate was set to be 0.08, and cells predicted to be doublets or with doubletScore larger than 0.25 were filtered. After quality control, a total of 1,598,708 cells were remained. 

## Data visualization

The stepwise quality control metrics used for indi- vidual samples were listed in Table S1. The resulting distribution of UMI counts, gene counts as well as mitochondrial gene percent- age were shown in Figures S1C–S1E. 

## Normalizaed UMI counts

We normalized the UMI counts with the deconvolution strategy implemented in the R package scran. Specifically, cell-specific size factors were computed by computeSumFactors function and further used to scale the counts for each cell. Then the logarithmic normalized counts were used for the downstream analysis.